In [1]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis  
from sklearn.neighbors import KNeighborsRegressor  
from sklearn.naive_bayes import GaussianNB  
from sklearn.tree import DecisionTreeClassifier  
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
# from sklearn import preprocessing

import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np


In [2]:
# загрузка данных
import tkinter.filedialog as tkFileDialog
def open_file():
    return tkFileDialog.askopenfilename( title='Выберите файл:')
data = pd.read_csv(open_file(), delimiter=";", encoding='cp1251', decimal=",") #delimiter=";", 
data = data.dropna()
convert_dict = { "Баллы": float }
data = data.astype(convert_dict)
data

,ФИО,Пол,Команда,Баллы,Сдал(-а),Посещение до Контрольная работа №1,Баллы до Контрольная работа №1,Контрольная работа №1,Посещение до Контрольная работа №2,Баллы до Контрольная работа №2,Контрольная работа №2,Посещение до Контрольная работа №3,Баллы до Контрольная работа №3,Контрольная работа №3
0,Абатуров Георгий Сергеевич,1.0,АЛГЕБРА Л-08,76.0,1,0.0,0.0,16,0.0,0.0,11.5,0.0,0.0,11.25
1,Абдимиталипов Мустафа,1.0,АЛГЕБРА Л-04,61.0,1,0.82,37.25,12,1.0,0,12,1.0,0,8
2,Абдимуминов Абдужалол Абдихаким угли,1.0,АЛГЕБРА Л-05,0.0,0,0.36,29.0,0,0.0,0,0,0.25,0,0
3,Абдразакова Екатерина Наилевна,0.0,АЛГЕБРА Л-04,100.0,1,0.82,8.0,20,0.6,9.0,20,1.0,7.0,20
4,Абдрашитова Диана Дамировна,0.0,АЛГЕБРА Л-07,61.0,1,0.73,17.0,17,0.8,0.0,15,0.88,0.0,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
947,Ярмоц Надежда Андреевна,0.0,АЛГЕБРА Л-08,66.5,1,1.0,30.0,7.5,1.0,0,9.5,1.0,0,14
948,Ярославцев Дмитрий Сергеевич,1.0,АЛГЕБРА Л-04,74.0,1,0.91,35.5,20,1.0,3.0,20,1.0,0.0,5
949,Яруллин Тимофей Русланович,1.0,АЛГЕБРА Л-08,76.0,1,0.91,30.0,1,1.0,2.0,20,0.88,4.0,15
950,Ятло Николай Олегович,1.0,АЛГЕБРА Л-04,19.0,0,0.82,30.0,9,0.8,0,8,0.38,0,2


In [3]:
SVC_model = SVC(kernel='rbf')
KNN_model = KNeighborsRegressor(n_neighbors=2)
Linear_discriminant = LinearDiscriminantAnalysis()
Gaussian_NB = GaussianNB()
DecisionTree_Classifier = DecisionTreeClassifier(random_state=0)
Logistic_Regression = LogisticRegression(random_state=0)

models = {"Метод опорных векторов": SVC_model, \
            "k-ближайших соседей": KNN_model, \
            "Линейный дискриминантный анализ": Linear_discriminant, \
            "Наивный метод Байеса": Gaussian_NB,  \
            "Дерево решений": DecisionTree_Classifier,  \
            "Логистическая регрессия": Logistic_Regression}

In [4]:
def prediction_regres(X, Y):
    Y=[int(i*100) for i in Y]
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.10, random_state=27)
    # y_train_original, y_train  = y_train, list(map(float, labels.fit_transform(y_train)))
    # y_test_original, y_test  = y_test, list(map(float, labels.fit_transform(y_test)))
    res = {}

    for key, model in models.items():
        model.fit(X_train, y_train)
        data_prediction = model.predict(X_test)

        res[key] = round(accuracy_score(data_prediction.round(), y_test), 3)
        data_prediction=[i/100.0 for i in data_prediction]
        # data_prediction = [y_train_original[list(y_train).index(i)] for i in data_prediction]
        # print(accuracy_score(data_prediction, y_test))  
    return res


def prediction_class(X, Y):
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.10, random_state=27)
    res = {}
    for key, model in models.items():
        model.fit(X_train, y_train)
        data_prediction = model.predict(X_test)

        res[key] = round(accuracy_score(data_prediction.round(), y_test), 3)
        # print(accuracy_score(data_prediction, y_test))  
    return res

        
        # print(classification_report(data_prediction, y_test))

In [5]:
exams = [name  for name in data.columns.to_list() if name.startswith("Контрольная")]
scores = [name  for name in data.columns.to_list() if name.startswith("Баллы ")]
being = [name  for name in data.columns.to_list() if name.startswith("Посещение ")]

convert_dict = {name: float for name in being+scores+exams+["Сдал(-а)"]}
# convert_dict["Сдал(-а)"] = int
data = data.astype(convert_dict)

not_for_prediction = ["ФИО", "Команда", "Направление", "Сдал(-а)", "Пол", "Оценка", "Баллы", "Повышение оценки"]

In [6]:
# определение оценки
data["Оценка"] = 0
data.loc[data["Баллы"]<61.0, "Оценка"] = 2 
data.loc[data["Баллы"]>=91.0, "Оценка"] = 5
data.loc[(data["Баллы"]<91) & (data["Баллы"]>=76.0), "Оценка"] = 4
data.loc[(data["Баллы"]<76.0) & (data["Баллы"]>=61.0), "Оценка"] = 3

data.head

<bound method NDFrame.head of                                       ФИО  Пол       Команда  Баллы  Сдал(-а)  \
0              Абатуров Георгий Сергеевич  1.0  АЛГЕБРА Л-08   76.0       1.0   
1                   Абдимиталипов Мустафа  1.0  АЛГЕБРА Л-04   61.0       1.0   
2    Абдимуминов Абдужалол Абдихаким угли  1.0  АЛГЕБРА Л-05    0.0       0.0   
3          Абдразакова Екатерина Наилевна  0.0  АЛГЕБРА Л-04  100.0       1.0   
4             Абдрашитова Диана Дамировна  0.0  АЛГЕБРА Л-07   61.0       1.0   
..                                    ...  ...           ...    ...       ...   
947               Ярмоц Надежда Андреевна  0.0  АЛГЕБРА Л-08   66.5       1.0   
948          Ярославцев Дмитрий Сергеевич  1.0  АЛГЕБРА Л-04   74.0       1.0   
949            Яруллин Тимофей Русланович  1.0  АЛГЕБРА Л-08   76.0       1.0   
950                 Ятло Николай Олегович  1.0  АЛГЕБРА Л-04   19.0       0.0   
951                             Яхья Исам  1.0  АЛГЕБРА Л-05   76.0       1.0  

In [7]:
# повысил ли студент оценку на экзамене
data["Повышение оценки"] = 0
for index, row in data.iterrows():
    res = row[scores+exams].sum() 
    en = row["Баллы"]
    # row["Повышение оценки"] = -1 if res<row["Баллы"] else 1 if res>row["Баллы"] else 0
    data.loc[index, "Повышение оценки"] = -1 if res<row["Баллы"] else 1 if res>row["Баллы"] else 0

print(f"Повысили баллы на экзамене {round(len(data.loc[data['Повышение оценки']==1, 'Баллы'])/len(data)*100, 2)}%")
print(f"Понизили баллы на экзамене {round(len(data.loc[data['Повышение оценки']==-1, 'Баллы'])/len(data)*100, 2)}%")
print(f"Не изменили баллы на экзамене {round(len(data.loc[data['Повышение оценки']==0, 'Баллы'])/len(data)*100, 2)}%")


Повысили баллы на экзамене 47.95%
Понизили баллы на экзамене 47.11%
Не изменили баллы на экзамене 4.94%


In [8]:
# try:
# data.drop(["ФИО", "Команда"], axis=1, inplace=True)
print("Прогнозирование сдачи дисциплины")
results = pd.DataFrame(index=list(models.keys()), columns=exams)
for ex in exams:
    results[ex] = ""
    X = data[data.loc[:,:ex].columns.difference(not_for_prediction+[ex])] #[name  for name in data.columns.to_list() if ex in name and ex!=name]]
    Y = data["Сдал(-а)"].to_list()
    
    pred_res = prediction_class(X, Y)
    for key, val in pred_res.items():
        results[ex][key] = val

results
# except:
#     # data.drop(["Направление"], axis=1, inplace=True)
#     X = data[data.columns.difference(["Направление", "Сдал(-а)"])]
#     Y = data["Сдал(-а)"].to_list()

Прогнозирование сдачи дисциплины


,Контрольная работа №1,Контрольная работа №2,Контрольная работа №3
Метод опорных векторов,0.729,0.833,0.865
k-ближайших соседей,0.615,0.812,0.771
Линейный дискриминантный анализ,0.729,0.823,0.885
Наивный метод Байеса,0.729,0.812,0.875
Дерево решений,0.688,0.812,0.823
Логистическая регрессия,0.74,0.833,0.854


In [9]:
# прогнозирование контрольных без учета оценок
# try:
# data.drop(["ФИО", "Команда"], axis=1, inplace=True)
print("Прогнозирование контрольных без учета оценок")
results = pd.DataFrame(index=list(models.keys()), columns=exams)
for ex in exams:
    results[ex] = ""
    X = data[data.loc[:,:ex].columns.difference(not_for_prediction+[ex]+scores)] #[name  for name in data.columns.to_list() if ex in name and ex!=name]]
    Y = data[ex] #.to_list()
    pred_res = prediction_regres(X, Y)
    for key, val in pred_res.items():
        results[ex][key] = val
display(results)
# except:
#     # data.drop(["Направление"], axis=1, inplace=True)
#     X = data[data.columns.difference(["Направление", "Сдал(-а)"])]
#     Y = data["Сдал(-а)"].to_list()

Прогнозирование контрольных без учета оценок


,Контрольная работа №1,Контрольная работа №2,Контрольная работа №3
Метод опорных векторов,0.219,0.271,0.281
k-ближайших соседей,0.021,0.125,0.135
Линейный дискриминантный анализ,0.198,0.292,0.281
Наивный метод Байеса,0.083,0.167,0.094
Дерево решений,0.219,0.219,0.219
Логистическая регрессия,0.198,0.271,0.281


In [10]:
# прогнозирование контрольных учитывая оценку
# try:
# data.drop(["ФИО", "Команда"], axis=1, inplace=True)
print("Прогнозирование контрольных с учетом оценок")
for name, group in data.groupby("Оценка"):
    results = pd.DataFrame(index=list(models.keys()), columns=exams)
    for ex in exams:
        results[ex] = ""

        X = group[group.loc[:,:ex].columns.difference(not_for_prediction+[ex])] #[name  for name in data.columns.to_list() if ex in name and ex!=name]]
        Y = group[ex].to_list()
        
        pred_res = prediction_regres(X, Y)
        for key, val in pred_res.items():
            results[ex][key] = val
    
    print(name)
    display(results)
# except:
#     # data.drop(["Направление"], axis=1, inplace=True)
#     X = data[data.columns.difference(["Направление", "Сдал(-а)"])]
#     Y = data["Сдал(-а)"].to_list()

Прогнозирование контрольных с учетом оценок
2


,Контрольная работа №1,Контрольная работа №2,Контрольная работа №3
Метод опорных векторов,0.241,0.414,0.448
k-ближайших соседей,0.034,0.069,0.207
Линейный дискриминантный анализ,0.276,0.345,0.448
Наивный метод Байеса,0.276,0.103,0.31
Дерево решений,0.31,0.172,0.103
Логистическая регрессия,0.276,0.414,0.483


3


,Контрольная работа №1,Контрольная работа №2,Контрольная работа №3
Метод опорных векторов,0.0,0.083,0.083
k-ближайших соседей,0.042,0.083,0.042
Линейный дискриминантный анализ,0.0,0.083,0.167
Наивный метод Байеса,0.042,0.042,0.042
Дерево решений,0.0,0.083,0.083
Логистическая регрессия,0.0,0.083,0.125


4


,Контрольная работа №1,Контрольная работа №2,Контрольная работа №3
Метод опорных векторов,0.28,0.4,0.44
k-ближайших соседей,0.0,0.16,0.08
Линейный дискриминантный анализ,0.28,0.36,0.4
Наивный метод Байеса,0.2,0.24,0.2
Дерево решений,0.16,0.24,0.16
Логистическая регрессия,0.28,0.4,0.44


5


,Контрольная работа №1,Контрольная работа №2,Контрольная работа №3
Метод опорных векторов,0.611,0.556,0.722
k-ближайших соседей,0.167,0.278,0.556
Линейный дискриминантный анализ,0.667,0.556,0.778
Наивный метод Байеса,0.389,0.056,0.167
Дерево решений,0.333,0.333,0.611
Логистическая регрессия,0.667,0.556,0.778


In [11]:
# команды или направления
exams = [name  for name in data.columns.to_list() if name.startswith("Контрольная")]
results = pd.DataFrame(index=list(data.groupby("Команда").groups.keys()), \
                        columns=exams+["Процент сдачи"])
for command, group in data.groupby("Команда"):
    for ex in exams:
        results[ex][group["Команда"].iloc[0]] = group[ex].astype(float).mean()
    results["Процент сдачи"][group["Команда"].iloc[0]] = group["Сдал(-а)"].astype(float).mean()
results
    

,Контрольная работа №1,Контрольная работа №2,Контрольная работа №3,Процент сдачи
АЛГЕБРА Л-01,10.422897,8.586449,8.698598,0.803738
АЛГЕБРА Л-02,11.096639,9.663866,13.42605,0.731092
АЛГЕБРА Л-03,11.942857,12.404762,14.166667,0.790476
АЛГЕБРА Л-04,11.6375,8.83,11.235,0.66
АЛГЕБРА Л-05,12.855,11.69,12.25,0.7
АЛГЕБРА Л-06,10.658654,10.744231,11.177885,0.730769
АЛГЕБРА Л-07,6.174157,12.393258,8.708989,0.550562
АЛГЕБРА Л-08,10.432609,9.056522,9.319565,0.626087
АЛГЕБРА Л-09,9.666071,11.142857,9.904911,0.651786
